# Scraping data from MD Case Search

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, datetime, os
from pathlib import Path
import string

## Create folder with today's date
today = datetime.date.today()  
todaystr = today.isoformat()   
dir_path = os.getcwd()
save_dir = dir_path + "/" + todaystr
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

## Set the target save location and Firefox preferences to auto download csv files
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", save_dir)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv")

## Create the Firefox session
driver = webdriver.Firefox(firefox_profile=profile)



In [5]:
## Get past the disclaimer splash screen
driver.get("http://casesearch.courts.state.md.us/casesearch/inquiry-index.jsp")
check = driver.find_element_by_name("disclaimer")
check.click()
continue_button = driver.find_element_by_name("action")
continue_button.click()

assert "No results found." not in driver.page_source

NoSuchWindowException: Message: Browsing context has been discarded


In [102]:
## The search page won't let you search for all cases, need to do 26 searches (A-Z)
for last_initial in string.ascii_lowercase:
    ## Enter search criteria
    last_name = driver.find_element_by_name("lastName")
    last_name.clear()
    last_name.send_keys(last_initial)
    driver.find_element_by_xpath("//select[@name='partyType']/option[text()='Plaintiff']").click()
    driver.find_element_by_xpath("//input[@name='site' and @value='CIVIL']").click()
    driver.find_element_by_xpath("//input[@name='courtSystem' and @value='D']").click()
    driver.find_element_by_xpath("//select[@name='countyName']/option[text()='Charles County']").click()
    last_name = driver.find_element_by_name("filingStart")
    last_name.clear()
    last_name.send_keys("6/1/2019")  ## Turn to variable
    last_name = driver.find_element_by_name("filingEnd")
    last_name.clear()
    last_name.send_keys("9/30/2019")  ## Turn to variable
    driver.find_element_by_xpath("//input[@name='action' and @value='Search']").click()

    element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "CSV"))).click() ## need to add logic for when X isn't found
    driver.back()
    time.sleep(15)
    

TimeoutException: Message: 


# Concatenate csv exports to dataframe

In [6]:
os.chdir(save_dir)
os.getcwd()

'/Users/joshuahogge/Projects/tiredLandlords/2019-09-24'

In [25]:
import csv
import glob

# Adding a header to each file
path = r'/Users/joshuahogge/Projects/tiredLandlords/2019-09-24' # use your path
all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    with open(filename, newline='') as f:
        r = csv.reader(f)
        data = [line for line in r]
    with open(filename,'w',newline='') as f:
        w = csv.writer(f)
        w.writerow(['Case Number', 'Name', 'Date of Birth', 'Party Type', 'Court', 
                        'Case Type', 'Case Status', 'Filing Date', 'Case Caption'])
        w.writerows(data)

In [15]:
import pandas as pd
# Concatenate csv to single dataframe
df = pd.concat([pd.read_csv(f, index_col=0) for f in all_files])

In [16]:
df

,Name,Date of Birth,Party Type,Court,Case Type,Case Status,Filing Date,Case Caption
Case Number,,,,,,,,
D042CV19010029,"Ma, Stephany",NaN,Plaintiff,Charles County District Court,CNSC,Open,06/11/2019,Stephany Ma vs. Nneka Nickens et al.
D042CV19010924,Machado Investment Properties,NaN,Plaintiff,Charles County District Court,CNSC,Open,07/30/2019,Machado Investment Properties vs. Shirley Holl...
D042CV19011064,"Mack Courtney, Sonya K",NaN,Plaintiff,Charles County District Court,CNLC,Open,08/09/2019,Sonya Mack Courtney vs. Pamela Stevenson
D042CV19010766,"Madrid, Carlos",NaN,Plaintiff,Charles County District Court,CNLC,Open,07/22/2019,Carlos Madrid vs. Real Property Management Cap...
D042CV19011770,"Maduforo, Kennedy",NaN,Plaintiff,Charles County District Court,TORLC,Open,09/17/2019,Kennedy Maduforo vs. Dana Napier
...,...,...,...,...,...,...,...,...
D042CV19011008,Crown Asset Management,NaN,Plaintiff,Charles County District Court,CNLC,Open,08/05/2019,Crown Asset Management vs. BENJAMIN DAWNS
D042CV19011749,"Cruz, Luis",NaN,Plaintiff,Charles County District Court,CNLC,Open,09/19/2019,Luis Cruz vs. Jorje Bejar
D042CV19011019,"Cruz-morales, Luisa",NaN,Plaintiff,Charles County District Court,TORLC,Open,08/05/2019,Luisa Cruz-Morales vs. Maria Bommarito
